# Connection

In [3]:
!pip install findspark

In [4]:
import findspark
findspark.init() 

In [5]:
import pyspark
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
import s3fs

In [6]:
spark = SparkSession.builder.appName("Dummy").master('local').config("spark.driver.memory", "60g").config('spark.dirver.maxResultSize', '6g').getOrCreate()
print("session = ", spark.getActiveSession())
config_spark_s3_adv = {'access_id': 'JLTHDJMKGTSENNZLD5YC',
                        'access_key': 'gtlSwf2xuppeomzYaAN3OFpcJiWtgAGUWNU7KIvJ',
                        'impl': 'org.apache.hadoop.fs.s3a.S3AFileSystem',
                        'endpoint': 'https://obs.ru-moscow-1.hc.sbercloud.ru',
                        'spark.driver.maxResultSize':0,
                      }

spark._jsc.hadoopConfiguration().set("fs.s3a.impl", config_spark_s3_adv["impl"])
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", config_spark_s3_adv["endpoint"])
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", config_spark_s3_adv["access_id"])
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", config_spark_s3_adv["access_key"])
spark._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
spark.conf.set("spark.sql.adaptive.enabled", "false")

22/10/28 14:51:04 WARN Utils: Your hostname, ecs-test288 resolves to a loopback address: 127.0.1.1; using 10.11.10.114 instead (on interface eth0)
22/10/28 14:51:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/28 14:51:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


session =  <pyspark.sql.session.SparkSession object at 0x7f9dc4a005b0>


# Income target

In [17]:
# take 3 years

txstat = "s3a://okko-prod/okko/datasets/txstat/txstat_*_2022*.parquet"
txstat = spark.read.parquet(txstat)

In [18]:
txstat.createOrReplaceTempView("txstat")

In [19]:
## Платят

In [20]:
import datetime
current_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

q = """
SELECT
    user_uid,
    element_uid,
    price
FROM 
    (
        SELECT
            CAST(CONCAT(
                SUBSTRING(REPLACE(format_string('%16s', conv(CAST(user_uid AS BIGINT), 10, 16)), ' ', '0'), 1, 8),
                '-',
                SUBSTRING(REPLACE(format_string('%16s', conv(CAST(user_uid AS BIGINT), 10, 16)), ' ', '0'), 9, 4),
                '-',
                SUBSTRING(REPLACE(format_string('%16s', conv(CAST(user_uid AS BIGINT), 10, 16)), ' ', '0'), 13, 4),
                '-0000-000000000000'
                ) AS STRING) AS user_uid,
                element_uid,
                price,
                tx_type,
                consumption_mode 
                
        FROM txstat
        WHERE
            (tx_type = 'PURCHASE'
            AND consumption_mode ='SUBSCRIPTION'
            AND element_uid IS NOT NULL
            AND price != 0.0)
            or
            (tx_type = 'VIEW'
            AND consumption_mode ='SUBSCRIPTION'
            AND element_uid IS NOT NULL
            AND price != 0.0)
            or
            (tx_type='PURCHASE'
             AND consumption_mode ='RENT'
             AND element_uid IS NOT NULL
             AND price != 0.0)
             or
             (tx_type='PURCHASE'
             AND consumption_mode ='DTO'
             AND element_uid IS NOT NULL
             AND price != 0.0)
    )
"""

q_1 = spark.sql(q.format(
        date=current_date
    ))
q_1.show()

+--------------------+--------------------+-----+
|            user_uid|         element_uid|price|
+--------------------+--------------------+-----+
|00000000-105C-2EB...|b5f01d35-abaf-4e0...|799.0|
|00000000-105C-2C7...|b5f01d35-abaf-4e0...|799.0|
|00000000-105C-2F9...|6c61cac5-63ac-4b9...|399.0|
|00000000-0834-F54...|d2f3976f-db5c-400...|349.0|
|00000000-1059-C6F...|b4eba600-7067-440...| 99.0|
|00000000-0DB6-E58...|6c61cac5-63ac-4b9...|399.0|
|00000000-102B-FBD...|6c61cac5-63ac-4b9...| 14.0|
|00000000-105C-2A4...|811c1d12-e8b7-49e...|599.0|
|00000000-1027-97C...|ac4d0ba1-e1c0-451...|999.0|
|00000000-105C-23E...|811c1d12-e8b7-49e...|599.0|
|00000000-0F74-D8E...|6c61cac5-63ac-4b9...|399.0|
|00000000-0EEF-844...|811c1d12-e8b7-49e...|599.0|
|00000000-0144-59C...|9f64e56a-b527-4ee...|599.0|
|00000000-0FBF-0CB...|811c1d12-e8b7-49e...|599.0|
|00000000-0F4C-81D...|980cb546-9583-4a9...| 99.0|
|00000000-102E-ACB...|ac4d0ba1-e1c0-451...|999.0|
|00000000-105C-2DC...|b5f01d35-abaf-4e0...|799.0|


In [21]:
df_2021 = q_1.toPandas()

In [22]:
df_2021

,user_uid,element_uid,price
0,00000000-105C-2EBD-0000-000000000000,b5f01d35-abaf-4e0f-95b7-2acc4bdd51c5,799.0
1,00000000-105C-2C75-0000-000000000000,b5f01d35-abaf-4e0f-95b7-2acc4bdd51c5,799.0
2,00000000-105C-2F9F-0000-000000000000,6c61cac5-63ac-4b92-ba73-20239fbdb7dc,399.0
3,00000000-0834-F54B-0000-000000000000,d2f3976f-db5c-4009-8a05-668673f577a0,349.0
4,00000000-1059-C6F1-0000-000000000000,b4eba600-7067-440d-9820-d681e00ab580,99.0
...,...,...,...
37264864,00000000-019A-F73D-0000-000000000000,811c1d12-e8b7-49e1-91ab-2685ffd18493,599.0
37264865,00000000-0F00-F57C-0000-000000000000,496de8b2-b0e6-4f7d-95ce-76aea1b01903,299.0
37264866,00000000-1211-68A0-0000-000000000000,b5f01d35-abaf-4e0f-95b7-2acc4bdd51c5,799.0
37264867,00000000-020D-E7D8-0000-000000000000,98cf70d4-3fe6-4df5-b879-b4de0bc6f516,99.0


In [23]:
df_2021.shape

(37264869, 3)

In [24]:
len(df_2021['element_uid'].unique())

9630

In [25]:
len(df_2021['user_uid'].unique())

13924566

In [9]:
import pickle

In [26]:

with open('txstat_2022.pkl', 'wb') as f:
    pickle.dump(df_2021, f)

### Loading txstat 3 years

In [ ]:
import pickle

with open('txstat_2020.pkl', 'rb') as f:
    txstat_2020 = pickle.load(f)
    
with open('txstat_2021.pkl', 'rb') as f:
    txstat_2021 = pickle.load(f)
    
with open('txstat_2022.pkl', 'rb') as f:
    txstat_2022 = pickle.load(f)

# # content features from ./content_features_creation
# with open('elements_available.pkl', 'rb') as f:
#     elements_available = pickle.load(f)

# with open('el_uid_min_ts.pkl', 'rb') as f:
#     el_uid_min_ts = pickle.load(f)

In [ ]:
with open('RATINGS_OKKO_item_coldstart_dataset.pkl', 'rb') as f:
    full_df = pickle.load(f)

In [ ]:
data_2020 = txstat_2020.groupby('element_uid').agg('sum')
data_2021 = txstat_2021.groupby('element_uid').agg('sum')
data_2022 = txstat_2022.groupby('element_uid').agg('sum')

In [ ]:
data_2020.shape, data_2021.shape, data_2022.shape

In [ ]:
three_years_df = data_2020.append([data_2021, data_2022])

In [13]:
three_years_df

,price
element_uid,
002134e1-267c-418a-9e0f-4b5ee7fc111c,4509186.0
0023d642-4c86-4724-9d57-280a8f4e86fa,46021.0
00247d78-8a1e-43fd-ae3d-8d0e3f3151b9,945.0
00282f21-ea8e-4df3-9382-7a942822198b,82481.0
0029671a-ae6b-447a-b49a-3a210a779bff,662029.0
...,...
ffdb6728-7bc5-4e08-9c1c-e2c8643c3190,169421.0
ffe70cae-7c9f-4b21-bb28-1b820d1bb48c,1583.0
ffeb6d52-2062-4ec4-851d-a0255fe0fc5d,3490.0


In [14]:
three_years_df = three_years_df.groupby('element_uid').agg('sum')
three_years_df.shape

(15664, 1)

In [15]:
three_years_df

,price
element_uid,
0009995a-605d-4958-afac-2457b8445d2f,2788.0
000d6913-68a1-4135-9bad-9bcd71708b2e,436.0
001d1aeb-4599-46ae-bc78-e01638e1fd31,78451.0
001f4aeb-1b47-4ad5-8a8c-21b576acbadf,37412.0
001f9f8f-a238-41a1-a783-91c851a64bc3,55094.0
...,...
ffdb6728-7bc5-4e08-9c1c-e2c8643c3190,503740.0
ffe70cae-7c9f-4b21-bb28-1b820d1bb48c,10809.0
ffeb6d52-2062-4ec4-851d-a0255fe0fc5d,28655.0


In [16]:
full_df = full_df.merge(three_years_df,
                   on='element_uid',
                   how='inner')

In [18]:
full_df.shape

(4140, 20)

In [19]:
with open('income_target.pkl', 'wb') as f:
    pickle.dump(full_df, f)

In [ ]:
## фото 

In [6]:
import msgpack
import boto3
import awswrangler as wr
ACCESS_KEY='du3wAsX0uzEBBn4AdkFpCX4ZTXvSWFVG48GA4jbT'
SECRET_KEY='WA0NJOX0AFAY4K1J92XZ'

wr.config.s3_endpoint_url = 'https://obs.ru-moscow-1.hc.sbercloud.ru'

boto3.setup_default_session(
    region_name='ru-moscow',
    aws_access_key_id=SECRET_KEY,
    aws_secret_access_key=ACCESS_KEY,
)

In [30]:
path_catalogue = 's3://ml-okko-object-storage/catalogue/element_kino.csv'
catalogue = wr.s3.read_csv(path_catalogue, encoding="utf-8")

In [31]:
catalogue

,element_uid,kinopoiskid
0,0002b2da-7965-4804-8046-83bc45c36323,818778
1,0007e3e8-0333-4926-a36a-c73c5075d64d,798660
2,001357f9-55a1-4d6c-834e-29c55a2f7b34,181807
3,001dc985-ad6c-4e0d-9ab9-950210df3b96,NaN
4,001e2119-d5c0-43b2-ac7c-d72eb4152644,1172552
...,...,...
272478,ffeca5c8-b78c-4d41-b56c-b469587b5998,NaN
272479,ffeeb9cc-0121-46b2-b994-fde5e6f52385,258382
272480,fff49942-4b4d-4a75-ae42-64a0531b78e8,NaN
272481,fff801f2-f290-3cc9-8594-1ff3572d54e7,NaN


In [32]:
catalogue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272483 entries, 0 to 272482
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   element_uid  272483 non-null  object
 1   kinopoiskid  148145 non-null  object
dtypes: object(2)
memory usage: 4.2+ MB


## не платят

In [80]:
import datetime
current_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

q2 = """
        SELECT
            CAST(CONCAT(
                SUBSTRING(REPLACE(format_string('%16s', conv(CAST(user_uid AS BIGINT), 10, 16)), ' ', '0'), 1, 8),
                '-',
                SUBSTRING(REPLACE(format_string('%16s', conv(CAST(user_uid AS BIGINT), 10, 16)), ' ', '0'), 9, 4),
                '-',
                SUBSTRING(REPLACE(format_string('%16s', conv(CAST(user_uid AS BIGINT), 10, 16)), ' ', '0'), 13, 4),
                '-0000-000000000000'
                ) AS STRING) AS user_uid,
                element_uid,
                price,
                element_type,
                consumption_mode,
                tx_type, 
                user_subscription_start_ts, user_subscription_end_ts
        FROM txstat
        WHERE
            (tx_type = 'PURCHASE'
            AND consumption_mode ='SUBSCRIPTION'
            AND to_timestamp('{date}') BETWEEN to_timestamp(user_subscription_start_ts) AND to_timestamp(user_subscription_end_ts)
            AND element_uid IS NOT NULL
            AND price = 0.0)
            or
            (tx_type = 'VIEW'
            AND consumption_mode ='SUBSCRIPTION'
            AND to_timestamp('{date}') BETWEEN to_timestamp(user_subscription_start_ts) AND to_timestamp(user_subscription_end_ts)
            AND element_uid IS NOT NULL
            AND price = 0.0)
            or
            (tx_type='PURCHASE'
            AND consumption_mode ='RENT'
            AND element_uid IS NOT NULL
            AND price = 0.0)
            or
            (tx_type='PURCHASE'
            AND consumption_mode ='DTO'
            AND element_uid IS NOT NULL
            AND price = 0.0)
"""

q_2 = spark.sql(q2.format(
        date=current_date
    ))
q_2.show()

+--------------------+--------------------+-----+------------+----------------+--------+--------------------------+------------------------+
|            user_uid|         element_uid|price|element_type|consumption_mode| tx_type|user_subscription_start_ts|user_subscription_end_ts|
+--------------------+--------------------+-----+------------+----------------+--------+--------------------------+------------------------+
|00000000-0054-DC1...|09ce0983-3458-49d...|  0.0|       movie|             DTO|PURCHASE|                      null|                    null|
|00000000-017A-BBC...|26e1dad4-be17-448...|  0.0|       movie|             DTO|PURCHASE|                      null|                    null|
|00000000-0242-A2A...|09ce0983-3458-49d...|  0.0|       movie|             DTO|PURCHASE|                      null|                    null|
|00000000-0246-427...|525b664f-dd57-42f...|  0.0|       movie|             DTO|PURCHASE|                      null|                    null|
|00000000-024

In [81]:
q_2.count()

1721484

In [ ]:
df2 = q_2.toPandas()

In [ ]:
len(df2['element_uid'].unique())

In [ ]:
len(df2['user_uid'].unique())